## Neo4j Import of GraphRAG Result Parquet files
This notebook imports the results of the GraphRAG indexing process into the Neo4j Graph database for further processing, analysis or visualization.

### How does it work?
The notebook loads the parquet files from the output folder of your indexing process and loads them into Pandas dataframes. It then uses a batching approach to send a slice of the data into Neo4j to create nodes and relationships and add relevant properties. The id-arrays on most entities are turned into relationships.

All operations use `MERGE`, so they are idempotent, and you can run the script multiple times.

If you need to clean out the database, you can run the following statement
```
MATCH (n)
CALL { WITH n DETACH DELETE n } IN TRANSACTIONS OF 25000 ROWS;
```

In [1]:
GRAPHRAG_FOLDER="./output/20240911-103108/artifacts"

In [ ]:
! pip install neo4j

In [2]:
import pandas as pd
from neo4j import GraphDatabase
import time

In [17]:
NEO4J_URI="bolt://localhost"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="AP585576ice"
NEO4J_DATABASE="test"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [4]:
def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.
    Parameters: statement is the Cypher query to execute, df is the dataframe to import, and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0,total, batch_size):
        batch = df.iloc[start: min(start+batch_size,total)]
        result = driver.execute_query("UNWIND $rows AS value " + statement, 
                                      rows=batch.to_dict('records'),
                                      database_=NEO4J_DATABASE)
        print(result.summary.counters)
    print(f'{total} rows in { time.time() - start_s} s.')    
    return total

### Indexes and Constraints
Indexes in Neo4j are only used to find the starting points for graph queries, e.g. quickly finding two nodes to connect. Constraints exist to avoid duplicates, we create them mostly on id's of Entity types.

We use some Types as markers with two underscores before and after to distinguish them from the actual entity types.

The default relationship type here is `RELATED` but we could also infer a real relationship-type from the description or the types of the start and end-nodes.

* `__Entity__`
* `__Document__`
* `__Chunk__`
* `__Community__`
* `__Covariate__`

In [18]:
# create constraints, idempotent operation

statements = """
create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique;
create constraint document_id if not exists for (d:__Document__) require d.id is unique;
create constraint entity_id if not exists for (c:__Community__) require c.community is unique;
create constraint entity_id if not exists for (e:__Entity__) require e.id is unique;
create constraint entity_title if not exists for (e:__Entity__) require e.name is unique;
create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique;
create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique;
""".split(";")

for statement in statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)


create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique

create constraint document_id if not exists for (d:__Document__) require d.id is unique

create constraint entity_id if not exists for (c:__Community__) require c.community is unique

create constraint entity_id if not exists for (e:__Entity__) require e.id is unique

create constraint entity_title if not exists for (e:__Entity__) require e.name is unique

create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique

create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique


## Import Process
### Importing the Documents
We're loading the parquet file for the documents and create nodes with their ids and add the title property. We don't need to store text_unit_ids as we can create the relationships and the text content is also contained in the chunks.

In [19]:
doc_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_documents.parquet', columns=["id", "title"])
doc_df.head(2)

,id,title
0,43368cc493912556756c5e329beb8751,1.txt
1,4ec1cf21f5ef9d80f787daccb3f38e75,2.txt


In [20]:
# import documents
statement = """
MERGE (d:__Document__ {id:value.id})
SET d += value {.title}
"""

batched_import(statement, doc_df)

{'_contains_updates': True, 'labels_added': 9, 'nodes_created': 9, 'properties_set': 18}
9 rows in 0.10932278633117676 s.


9

### Loading Text Units
We load the text units, create a node per id and set the text and number of tokens. Then we connect them to the documents that we created before.

In [21]:
text_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_text_units.parquet',
                          columns=["id","text","n_tokens","document_ids"])
text_df.head(2)

,id,text,n_tokens,document_ids
0,49f2c2ae5443497c09490140bb6cd1f9,第4回 五行山从师取真经\n\n 五百年以后，观音菩萨奉了如来佛的法旨，带着锦袈裟等五...,300,[1603f328b4ed185c3925e1fee2b550a5]
1,3cebda9b412902bb448a08de15092bf0,来佛二弟子金蝉子转世的，观音暗中选定他为取经人，自己与惠岸行者变成了游方和尚，捧着袈裟等宝贝...,300,[1603f328b4ed185c3925e1fee2b550a5]


In [22]:
statement = """
MERGE (c:__Chunk__ {id:value.id})
SET c += value {.text, .n_tokens}
WITH c, value
UNWIND value.document_ids AS document
MATCH (d:__Document__ {id:document})
MERGE (c)-[:PART_OF]->(d)
"""

batched_import(statement, text_df)

{'_contains_updates': True, 'labels_added': 145, 'relationships_created': 145, 'nodes_created': 145, 'properties_set': 435}
145 rows in 0.24593043327331543 s.


145

### Loading Nodes
For the nodes we store id, name, description, embedding (if available), human readable id.

In [23]:
entity_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_entities.parquet',
                            columns=["name","type","description","human_readable_id","id","description_embedding","text_unit_ids"])
entity_df.head(2)

,name,type,description,human_readable_id,id,description_embedding,text_unit_ids
0,"""五行山""","""GEO""","The ""Five Elements Mountain,"" also known as ""W...",0,b45241d70f0e43fca764df95b2b81f77,"[-0.02117370069026947, -0.07958780229091644, 0...","[3d7b1af4625ae7ce28aee97b4905f6a2, 49f2c2ae544..."
1,"""观音菩萨""","""PERSON""","The entity ""Guanyin Bodhisattva,"" also known a...",1,4119fd06010c494caa07f439b333f4c5,"[-0.09430186450481415, -0.7348642945289612, 0....","[122240eec13dc4e3f2d7a6f5a9694749, 1c779f74d5f..."


In [24]:
entity_statement = """
MERGE (e:__Entity__ {id:value.id})
SET e += value {.human_readable_id, .description, name:replace(value.name,'"','')}
WITH e, value
CALL db.create.setNodeVectorProperty(e, "description_embedding", value.description_embedding)
CALL apoc.create.addLabels(e, case when coalesce(value.type,"") = "" then [] else [apoc.text.upperCamelCase(replace(value.type,'"',''))] end) yield node
UNWIND value.text_unit_ids AS text_unit
MATCH (c:__Chunk__ {id:text_unit})
MERGE (c)-[:HAS_ENTITY]->(e)
"""

batched_import(entity_statement, entity_df)

{'_contains_updates': True, 'labels_added': 369, 'relationships_created': 920, 'nodes_created': 369, 'properties_set': 1476}
369 rows in 2.451350688934326 s.


369

### Import Relationships
For the relationships we find the source and target node by name, using the base `__Entity__` type. After creating the RELATED relationships, we set the description as attribute.

In [25]:
rel_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_relationships.parquet',
                         columns=["source","target","id","rank","weight","human_readable_id","description","text_unit_ids"])
rel_df.head(2)

,source,target,id,rank,weight,human_readable_id,description,text_unit_ids
0,"""五行山""","""孙悟空""",bc70fee2061541148833d19e86f225b3,58,1.0,0,"""Sun Wukong was imprisoned in the Five Element...",[bd4161d66503cacdbfb59cc2b2b15888]
1,"""五行山""","""悟空""",0fc15cc3b44c4142a770feb4c037a6f7,96,2.0,1,"The entity ""五行山"" (Five-Heaven Mountain) is dir...","[3d7b1af4625ae7ce28aee97b4905f6a2, f2ae22fce21..."


In [26]:
rel_statement = """
    MATCH (source:__Entity__ {name:replace(value.source,'"','')})
    MATCH (target:__Entity__ {name:replace(value.target,'"','')})
    // not necessary to merge on id as there is only one relationship per pair
    MERGE (source)-[rel:RELATED {id: value.id}]->(target)
    SET rel += value {.rank, .weight, .human_readable_id, .description, .text_unit_ids}
    RETURN count(*) as createdRels
"""

batched_import(rel_statement, rel_df)

{'_contains_updates': True, 'relationships_created': 423, 'properties_set': 2538}
423 rows in 1.228727102279663 s.


423

### Importing Communities
For communities we import their id, title, level. We connect the `__Community__` nodes to the start and end nodes of the relationships they refer to.

Connecting them to the chunks they orignate from is optional, as the entites are already connected to the chunks.

In [27]:
community_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_communities.parquet', 
                     columns=["id","level","title","text_unit_ids","relationship_ids"])

community_df.head(2)

,id,level,title,text_unit_ids,relationship_ids
0,0,0,Community 0,"[3d7b1af4625ae7ce28aee97b4905f6a2,49f2c2ae5443...","[bc70fee2061541148833d19e86f225b3, 0fc15cc3b44..."
1,1,0,Community 1,"[122240eec13dc4e3f2d7a6f5a9694749,1c779f74d5f8...","[a24e9df02e1b4b43bf6324b039e28285, ab3a5a67132..."


In [28]:
statement = """
MERGE (c:__Community__ {community:value.id})
SET c += value {.level, .title}
/*
UNWIND value.text_unit_ids as text_unit_id
MATCH (t:__Chunk__ {id:text_unit_id})
MERGE (c)-[:HAS_CHUNK]->(t)
WITH distinct c, value
*/
WITH *
UNWIND value.relationship_ids as rel_id
MATCH (start:__Entity__)-[:RELATED {id:rel_id}]->(end:__Entity__)
MERGE (start)-[:IN_COMMUNITY]->(c)
MERGE (end)-[:IN_COMMUNITY]->(c)
RETURN count(distinct c) as createdCommunities
"""

batched_import(statement, community_df)

{'_contains_updates': True, 'labels_added': 58, 'relationships_created': 1171, 'nodes_created': 58, 'properties_set': 174}
58 rows in 2.161278009414673 s.


58

### Importing Community Reports
Fo the community reports we create nodes for each communitiy set the id, community, level, title, summary, rank, and rank_explanation and connect them to the entities they are about. For the findings we create the findings in context of the communities.

In [29]:
community_report_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_community_reports.parquet',
                               columns=["id","community","level","title","summary", "findings","rank","rank_explanation","full_content"])
community_report_df.head(2)

,id,community,level,title,summary,findings,rank,rank_explanation,full_content
0,7cc91019-b9b7-4d23-a93c-f5f93aea0a19,48,2,Monkey King's Journey Community,The Monkey King's Journey Community is centere...,"[{'explanation': 'Sun Wukong, the Monkey King,...",7.5,The impact severity rating is high due to the ...,# Monkey King's Journey Community\n\nThe Monke...
1,98537d67-3419-4709-9582-2c745928501f,49,2,Monsters and Mythical Entities Community,The Monsters and Mythical Entities Community i...,[{'explanation': 'The Demon King's monsters ar...,4.5,The impact severity rating is moderate due to ...,# Monsters and Mythical Entities Community\n\n...


In [30]:
# import communities
community_statement = """
MERGE (c:__Community__ {community:value.community})
SET c += value {.level, .title, .rank, .rank_explanation, .full_content, .summary}
WITH c, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH c, value, finding_idx, value.findings[finding_idx] as finding
MERGE (c)-[:HAS_FINDING]->(f:Finding {id:finding_idx})
SET f += finding
"""
batched_import(community_statement, community_report_df)

{'_contains_updates': True, 'labels_added': 277, 'relationships_created': 277, 'nodes_created': 277, 'properties_set': 1155}
54 rows in 0.30412888526916504 s.


54

### Importing Covariates
Covariates are for instance claims on entities, we connect them to the chunks where they originate from.

**By default, covariates are not included in the output, so the file might not exists in your output if you didn't set the configuration to extract claims**

In [39]:
"""
# cov_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_covariates.parquet')
# cov_df.head(2)
"""

"\n# cov_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_covariates.parquet')\n# cov_df.head(2)\n"

In [40]:
cov_statement = """
MERGE (c:__Covariate__ {id:value.id})
SET c += apoc.map.clean(value, ["text_unit_id", "document_ids", "n_tokens"], [NULL, ""])
WITH c, value
MATCH (ch:__Chunk__ {id: value.text_unit_id})
MERGE (ch)-[:HAS_COVARIATE]->(c)
"""
# batched_import(cov_statement, cov_df)